In [ ]:
import requests
from bs4 import BeautifulSoup
import json

link = 'http://m.endic.naver.com/example.nhn?sLn=kr&exampleId=%s&webCrawl=0'
link_n = 6540

data = []


for i in range(link_n):
    url = link % (i+1)
    response = requests.get(url).text
    soup = BeautifulSoup(response, 'html.parser')
    korean = str(soup.find_all(class_="text_wrap")[0])[24:-7]
    eng = str(soup.find_all(class_="cp_trans_area")[0])[27:-6]
    if i % 10 == 0:
        print((korean, eng))
        with open('data.txt', 'w') as outfile:
            json.dump(data, outfile)
    data.append((korean, eng))

In [3]:
import json
data = json.load(open("data.txt", "r"))


In [4]:
N = len(data)
print(N)
kor, eng = [], []
for x in data:
    kor.append(x[0])
    _eng = []
    for word in x[1].split():
        word = word.lower()
        punc = None
        if len(word) > 1:
            if ord(word[-1]) < ord('a') or ord(word[-1]) > ord('z'):
                punc = word[-1]
                word = word[:-1]
        _eng.append(word)
        if punc: _eng.append(punc)
    eng.append(_eng)

print(eng[0])

6540
['how', 'are', 'you', 'doing', '?']


In [5]:
dec_wtoi = {}
dec_itow = []
m_dec = 0

enc_wtoi = {}
enc_itow = []
m_enc = 0

l_enc = 0


for words in kor:
    cnt = 0
    l_enc = max(l_enc, len(words))
    for char in words:
        if char not in enc_wtoi:
            enc_wtoi[char] = m_enc
            enc_itow.append(char)
            m_enc += 1
    
enc_wtoi[' '] = m_enc
enc_itow.append(' ')

dec_wtoi['<START>'] = m_dec
dec_itow.append('<START>')
m_dec += 1

l_dec = 0
for words in eng:
    l_dec = max(l_dec, len(words))
    for word in words:
        if word not in dec_wtoi:
            dec_wtoi[word] = m_dec
            dec_itow.append(word)
            m_dec += 1
        
dec_wtoi['<END>'] = m_dec
dec_itow.append('<END>')
m_dec += 1

print ("l_enc: ", l_enc)
print ("l_dec: ", l_dec)
print ("m_enc: ", m_enc)
print ("m_dec: ", m_dec)

l_enc:  75
l_dec:  33
m_enc:  1064
m_dec:  3719


In [6]:
import tensorflow as tf
import numpy as np

In [7]:
X_enc_train = m_enc * np.ones((N, l_enc))
X_dec_train = (m_dec - 1) * np.ones((N, l_dec+1))
y_train = (m_dec - 1) * np.ones((N, l_dec+1))
for i, word in enumerate(kor):
    cnt = 0
    for j, char in enumerate(word):
        if char == ' ': continue
        cnt += 1
        X_enc_train[i, l_enc-cnt] = enc_wtoi[char]

for i, words in enumerate(eng):
    X_dec_train[i][0] = 0
    for j, word in enumerate(words):
        X_dec_train[i, j+1] = y_train[i, j] = dec_wtoi[word]

print(X_enc_train)
print(X_dec_train)
print(y_train)

[[1.064e+03 1.064e+03 1.064e+03 ... 2.000e+00 1.000e+00 0.000e+00]
 [1.064e+03 1.064e+03 1.064e+03 ... 1.000e+01 4.000e+00 9.000e+00]
 [1.064e+03 1.064e+03 1.064e+03 ... 1.900e+01 1.800e+01 1.700e+01]
 ...
 [1.064e+03 1.064e+03 1.064e+03 ... 1.600e+01 3.530e+02 0.000e+00]
 [1.064e+03 1.064e+03 1.064e+03 ... 1.600e+01 3.530e+02 0.000e+00]
 [1.064e+03 1.064e+03 1.064e+03 ... 3.000e+01 1.600e+01 4.170e+02]]
[[0.000e+00 1.000e+00 2.000e+00 ... 3.718e+03 3.718e+03 3.718e+03]
 [0.000e+00 6.000e+00 7.000e+00 ... 3.718e+03 3.718e+03 3.718e+03]
 [0.000e+00 1.200e+01 1.300e+01 ... 3.718e+03 3.718e+03 3.718e+03]
 ...
 [0.000e+00 2.240e+02 2.110e+02 ... 3.718e+03 3.718e+03 3.718e+03]
 [0.000e+00 6.590e+02 2.110e+02 ... 3.718e+03 3.718e+03 3.718e+03]
 [0.000e+00 3.500e+01 5.210e+02 ... 3.718e+03 3.718e+03 3.718e+03]]
[[1.000e+00 2.000e+00 3.000e+00 ... 3.718e+03 3.718e+03 3.718e+03]
 [6.000e+00 7.000e+00 8.000e+00 ... 3.718e+03 3.718e+03 3.718e+03]
 [1.200e+01 1.300e+01 1.400e+01 ... 3.718e+03 3.71

In [8]:
learning_rate = 1e-3
n_hidden = 200

tf.reset_default_graph()

X_enc = tf.placeholder(tf.int32, [None, l_enc])
X_dec = tf.placeholder(tf.int32, [None, None])
y = tf.placeholder(tf.int32, [None, None])
is_training = tf.placeholder(tf.bool)

def model(X_enc, X_dec, is_training):
    keep_prob = tf.cond(is_training, lambda:tf.constant(0.5), lambda:tf.constant(1.0))
    if is_training is not None:
        trainable = True
    else:
        trainable = False
    embedding_enc = tf.get_variable('embedding_enc', initializer=tf.random_uniform([m_enc, n_hidden], -1.0, 1.0), trainable=trainable)
    mt = tf.get_variable('mt', shape=[1, n_hidden], initializer=tf.zeros_initializer(), trainable=False, dtype=tf.float32)
    embedding_enc = tf.concat([embedding_enc, mt], 0, name='concatt')
    embedding_dec = tf.get_variable('embedding_dec', initializer=tf.random_uniform([m_dec, n_hidden], -1.0, 1.0), trainable=trainable)
    after_embedding_enc = tf.cast(tf.nn.embedding_lookup(embedding_enc, X_enc, validate_indices=False), tf.float32)
    after_embedding_dec = tf.cast(tf.nn.embedding_lookup(embedding_dec, X_dec, validate_indices=False), tf.float32)
    
    inputs = tf.layers.conv2d(inputs=tf.reshape(after_embedding_enc, [-1, l_enc, n_hidden, 1]),
                              filters=1,
                              kernel_size=(1, 5),
                              trainable=trainable)
    
    with tf.variable_scope('encode'):
        enc_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
        enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=keep_prob)
        outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, after_embedding_enc, dtype=tf.float32)
    
    with tf.variable_scope('decode'):
        dec_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
        dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=keep_prob)
        outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, after_embedding_dec, initial_state=enc_states, dtype=tf.float32)
    
    out = tf.layers.dense(outputs, n_hidden, activation=None, trainable=trainable)
    out = tf.tensordot(out, embedding_dec, axes=[[2], [1]])
    print(out.get_shape())
    return out

y_out = model(X_enc, X_dec, is_training)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_out, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

(?, ?, 3719)


In [12]:
from IPython.display import clear_output
import random

batch_size = 128
test_num = 1000
N_ = N - test_num

indices = np.arange(N)
np.random.shuffle(indices)
test_indices = indices[-test_num:]
train_indices = indices[:-test_num]


sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

stp = 0
best_counter = 0
for epoch in range(1000):
    # train
    epoch_indices = np.arange(N_)
    np.random.shuffle(epoch_indices)
    th = 0
    while th < N_:
        rng = train_indices[epoch_indices[th:min(th+batch_size, N_)]]
        feed_dict = {X_enc: X_enc_train[rng],
                     X_dec: X_dec_train[rng],
                     is_training: True,
                     y: y_train[rng]}
        th += batch_size
        loss, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        stp += 1
        
    clear_output()
    print ('Epoch #%d' % (epoch+1))
    print ('train_loss:', loss)
    
    #validation
    correct = 0
    for k in range(0, test_num, 200):
        y_now = np.zeros((200, 1))
        rng = test_indices[k:k+200]
        for i in range(1, l_dec):
            feed_dict = {X_enc: X_enc_train[rng],
                         X_dec: X_dec_train[rng],
                         is_training: False}
            res = np.array(sess.run(y_out, feed_dict=feed_dict))
            y_now = np.argmax(res, axis=2)

        feed_dict = {X_enc: X_enc_train[rng],
                     X_dec: X_dec_train[rng],
                     y: y_train[rng],
                     is_training: False}
        loss, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        print ('val_loss:', loss)

        for ii in range(200):
            i = test_indices[k + ii]
            import random
            print_mode = random.random() < 0.005
            if print_mode:
                print (" input: ", kor[i])
                print ("answer: ", " ".join(eng[i]))
                print (" model: ", end=" ")
            model_answer = []
            for j in range(l_dec):
                if y_now[ii, j] == m_dec - 1:
                    break
                model_answer.append(dec_itow[y_now[ii, j]])
            model_answer = " ".join(model_answer)
            if model_answer == " ".join(eng[i]):
                correct += 1
            if print_mode:
                print(model_answer, end='\n\n')
    print("correct: ", correct)
    if correct >= best_counter:
        best_counter = correct
        saver.save(sess, "./model_large.ckpt")
        
    """
    for ii in range(200):
        i = np.random.randint(0, N)
        print (" input: ", kor[i])
        print ("answer: ", " ".join(eng[i]))
        print (" model: ", end=" ")
        y_now = [0]
        model_answer = []
        for j in range(l_dec):
            res = np.array(sess.run(y_out, feed_dict={X_enc: X_enc_train[i:i+1],
                                                      X_dec: np.array([y_now]),
                                                      is_training: False}))
            best_word_idx = np.argmax(res[0][j])
            y_now.append(best_word_idx)
            if best_word_idx == m_dec - 1:
                break
            model_answer.append(dec_itow[np.argmax(res[0][j])])
        model_answer = " ".join(model_answer)
        if model_answer == " ".join(eng[i]):
            model_answer += ' <- and the answer is correct!'
        print(model_answer, end='\n\n')
    """

Epoch #100
train_loss: 0.06737235
val_loss: 0.02902684
 input:  곧 가지 않으면 안됩니다.
answer:  i'm in a hurry to go .
 model:  i'm in a hurry to go .

val_loss: 0.03054464
 input:  그런 식으로 말하면 안돼요.
answer:  you shouldn't say things like that .
 model:  you shouldn't say things like that .

 input:  당신 말도 일리는 있지만.
answer:  you have a point , but .
 model:  you have a point , but .

val_loss: 0.031544395
val_loss: 0.031003462
 input:  아무라도 바꿔 주세요.
answer:  i may speak to anybody .
 model:  i may speak to anybody .

val_loss: 0.02584194
correct:  844


KeyboardInterrupt: 

In [46]:
print(np.setdiff1d(indices[train_indices], indices[test_indices]).shape == train_indices.shape)
print(np.setdiff1d(indices[test_indices], indices[train_indices]).shape == test_indices.shape)

# 즉, train data와 validation data는 분리되어 있다.

True
True
